<a href="https://colab.research.google.com/github/Surya-teja-888/IPL-Dataset-Player-price-prediction/blob/master/Stroke_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
upload = files.upload()

Saving stroke_detection.csv to stroke_detection.csv


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras import layers
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("stroke_detection.csv")
data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


Removing "id" from the data

In [4]:
data_1 = data.drop("id",axis = 1)
data_1.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


Checking For NaN values

In [5]:
data.isnull().sum() # We have 201 NaN values in "bmi" column

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

Replacing NaN values

In [6]:
# Replacing NaN values with mean of the "bmi" column
data["bmi"].mean()

28.893236911794673

In [9]:
data_1["bmi"] = data_1.fillna(data["bmi"].mean()) # We replaced NaN values with mean of the "bmi" column

In [10]:
data_1.isnull().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

Now, Encoding Categorical Features

In [12]:
x_features = data_1.columns
encoded_data = pd.get_dummies(data_1[x_features],drop_first = True)
encoded_data

,age,hypertension,heart_disease,avg_glucose_level,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,bmi_Male,bmi_Other,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,1,1,0,1,0,1,0,0,1,1,0,1,0,0
1,61.0,0,0,202.21,1,0,0,1,0,0,1,0,0,0,0,0,1,0
2,80.0,0,1,105.92,1,1,0,1,0,1,0,0,0,1,0,0,1,0
3,49.0,0,0,171.23,1,0,0,1,0,1,0,0,1,0,0,0,0,1
4,79.0,1,0,174.12,1,0,0,1,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,80.0,1,0,83.75,0,0,0,1,0,1,0,0,1,0,0,0,1,0
5106,81.0,0,0,125.20,0,0,0,1,0,0,1,0,1,0,0,0,1,0
5107,35.0,0,0,82.99,0,0,0,1,0,0,1,0,0,0,0,0,1,0
5108,51.0,0,0,166.29,0,1,0,1,0,1,0,0,0,1,0,1,0,0


In [13]:
x = encoded_data.loc[:,encoded_data.columns != "smoking_status_smokes"]
y = encoded_data["smoking_status_smokes"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 42)

Building Neural Network

In [14]:
x_train.shape

(3577, 17)

In [22]:
# Standardization of the data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [18]:
model = keras.Sequential([
                          layers.Dense(32,activation = "relu",name = "layer1"),
                          layers.Dense(32,activation = "relu",name = "layer2"),
                          layers.Dense(1,activation = "sigmoid",name = "layer3"),
])

In [24]:
# Compiling the model
model.compile(optimizer="Adam",loss = "binary_crossentropy",metrics = ["accuracy"])
model.fit(x_train,y_train,validation_split = 0.3,batch_size = 100,epochs = 100) 

Epoch 1/100
26/26 [==============================] - 1s 9ms/step - loss: 0.2199 - accuracy: 0.8695 - val_loss: 0.2168 - val_accuracy: 0.8613
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2143 - accuracy: 0.8758 - val_loss: 0.2154 - val_accuracy: 0.8641
Epoch 3/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2271 - accuracy: 0.8692 - val_loss: 0.2166 - val_accuracy: 0.8529
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2254 - accuracy: 0.8680 - val_loss: 0.2152 - val_accuracy: 0.8575
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2239 - accuracy: 0.8648 - val_loss: 0.2169 - val_accuracy: 0.8585
Epoch 6/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2229 - accuracy: 0.8712 - val_loss: 0.2150 - val_accuracy: 0.8622
Epoch 7/100
26/26 [==============================] - 0s 3ms/step - loss: 0.2158 - accuracy: 0.8631 - val_loss: 0.2142 - val_accuracy: 0.8613
Epoch 8/100
2

In [26]:
y_pred = model.predict(x_test)
y_pred.shape

(1533, 1)

In [29]:
y_pred = y_pred > 0.5

In [30]:
metrics.accuracy_score(y_test,y_pred)

0.8408349641226354